<a href="https://colab.research.google.com/github/Rajeevku1/AIAgentSamples/blob/main/MeetingMinutesAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -q --upgrade bitsandbytes accelerate

In [8]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata


In [9]:
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

# STEP 1: Transcribe Audio

## Option 1: Use Open Source for Transcription - Hugging Face Pipelines

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-medium.en",
    dtype=torch.float16,
    device='cuda',
    return_timestamps=True
)

result = pipe(audio_filename)
transcription = result["text"]
print(transcription)

## Option 2: Use OpenAI for Transcription

In [11]:
# Sign in to OpenAI using Secrets in Colab

AUDIO_MODEL = "gpt-4o-mini-transcribe"
audio_filename = "/tmp/audio.m4a"
# Open the file
audio_file = open(audio_filename, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

I wanted to report a problem regarding my card which has been lost. I want to make sure you can lock my card so that there is no fraud transaction happen.



In [12]:
display(Markdown(transcription))
print("\n\n")
display(Markdown(transcription))

I wanted to report a problem regarding my card which has been lost. I want to make sure you can lock my card so that there is no fraud transaction happen.


I wanted to report a problem regarding my card which has been lost. I want to make sure you can lock my card so that there is no fraud transaction happen.


# STEP 2: Analyze & Report

In [16]:
system_message = """
You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.
"""

user_prompt = f"""
Below is an extract transcript of a RCLI Project demo meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
{transcription}
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [17]:
openai_response = openai.chat.completions.create(model="gpt-4o-mini",messages=messages, temperature=0.7)
meeting_summary = openai_response.choices[0].message.content
print("\n--- OpenAI Meeting Summary ---")
display(Markdown(meeting_summary))


--- OpenAI Meeting Summary ---


# Minutes of RCLI Project Demo Meeting

**Date:** [Insert Date]  
**Location:** [Insert Location]  
**Attendees:** [Insert Attendees]

## Summary
The meeting focused on discussing issues related to card management, specifically the loss of a card and the need for security measures to prevent fraudulent transactions.

## Discussion Points
- A participant reported a lost card and requested the ability to lock it to prevent unauthorized transactions.
- The importance of card security and immediate response to loss was emphasized.

## Takeaways
- The necessity for a system feature that allows users to lock their cards instantly in the event of loss.

## Action Items
- **[Owner Name]**: Investigate and implement a feature for users to lock their cards immediately.
- **[Owner Name]**: Create a communication plan to inform users about the process of reporting lost cards and locking them.

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

In [ ]:
response = tokenizer.decode(outputs[0])

In [ ]:
display(Markdown(response))

# Student contribution

Student Emad S. has made this powerful variation that uses `TextIteratorStreamer` to stream back results into a Gradio UI, and takes advantage of background threads for performance! I'm sharing it here if you'd like to take a look at some very interesting work. Thank you, Emad!

https://colab.research.google.com/drive/1Ja5zyniyJo5y8s1LKeCTSkB2xyDPOt6D